# Some notes on working with Z3

First, be careful.  There is a z3 package that is not the solver.  When you perform an installation, you need to make sure to install the `z3-solver` package.

```$ pip3 install z3-solver```

If you are running in a conda environment, you might want to use conda to install Z3 instead.  As usual pip will be fast and conda will likely take a long time.

```$ conda install -c danielbok z3-solver```

You should be able to interact with Z3 at the prompt, and you can use Z3 from inside Python scripts by including the package in the usual way.

In [1]:
# Install the Z3 package in the current Jupyter kernel if it is not already.
import sys
!{sys.executable} -m pip install z3-solver

In [2]:
from z3 import *

## Tutorials

You can read about how to use Z3 online.  A nice, quick introduction, and the one I would suggest, can be found [here](https://ericpony.github.io/z3py-tutorial/guide-examples.htm).  A longer detailed tutorial can be found [here](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.225.8231&rep=rep1&type=pdf).  Even if you aren't doing code reverse engineering, Z3 is an incredibly useful tool.

Want to watch a one-hour video about Z3?  [Here you go](https://www.youtube.com/watch?v=TgAVIqraCHo).

## Variables

Create variables using the various type constructors.  For what you are doing, you will need `BitVec`.  Be aware that you may also need to create *values*, and that 5 is not the same as a 64-bit 5.  For values you can use `BitVecVal`.  Note that there are plural versions of the variable constructors that can construct multiple variables at once.  An example is below.

In [4]:
rax, rbx, rcx, rdx, rsi, rdi, rbp = BitVecs('rax rbx rcx rdx rsi rdi rbp', 64)

If you want to use values, you need to create bit vectors.  For example, to create a 64-bit twos-complement version of -5 as a bit vector, use `BitVecVal(-5,64)`.

In [5]:
m5 = BitVecVal(-5,64)
print(m5.sexpr())

#xfffffffffffffffb


As noted in the short tutorial linked above, "In Z3Py, the operators <, <=, >, >=, /, % and >> correspond to the signed versions. The corresponding unsigned operators are ULT, ULE, UGT, UGE, UDiv, URem and LShR."  Be careful which version you use!

## Solve

Z3 is incredibly useful because it understands twos-complement arithmetic.

In [6]:
x = BitVec('x',16)
solve(x == -x, x != 0)

[x = 32768]


## Assembly to Z3

Suppose I have the following assembly listing.

```asm
    mov rax, rdi
    lea rbx, [rax*2+20]
    add rax, rbx
    asl rax, 1
    cmp eax, 0x1000
```

How can we represent what it happenning?  Well, first, we can rewrite this in SSA form.

```asm
    mov rax_1, rdi_0
    lea rbx_1, [rax_1*2+20]
    add rax_2, rbx_1
    asl rax_3, 1
    cmp eax_3, 0x1000
```

Let's create the various variables we will need.

In [7]:
rax_1, rax_2, rax_3 = BitVecs('rax_1 rax_2 rax_3', 64)
rdi_0 = BitVec('rdi_0', 64)
rbx_1 = BitVec('rbx_1', 64)

Now we can represent the semantics of each line in Z3.

In [9]:
s = Solver()
s.add(rax_1 == rdi_0)               # mov rax_1, rdi_0
s.add(rbx_1 == rax_1 * 2 + 20)      # lea rbx_1, [rax_1*2+20]
s.add(rax_2 == rax_1 + rbx_1)       # add rax_2, rbx_1
s.add(rax_3 == rax_2 << 1)          # asl rax_3, 1
mask32 = BitVecVal(0xffffffff, 64)
s.add(rax_3 & mask32 == 0x1000)     # cmp eax_3, 0x1000

Now we can check to see if there is a solution in which the last comparison succeeds (the values are equal).

In [10]:
s.check()

sat

There is!  Let's get the solution.  We really care about `rdi_0`, since that's the only input.

In [11]:
s.model()

[rax_1 = 676,
 rax_3 = 4096,
 rax_2 = 2048,
 rbx_1 = 1372,
 rdi_0 = 676]

Does this value (676) work?

In [12]:
676 * 2 + 20

1372

In [13]:
1372 + 676

2048

In [14]:
2048 << 1

4096

In [15]:
4096 == 0x1000

True

It does!  I hope this helps.  Send me questions you have.